<a href="https://colab.research.google.com/github/areebarshad/sp500-sector-prediction/blob/main/notebooks/LGBM_model_functions/LGBM_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#define a feature engineering function for the sector pipeline
def generate_features(returns, target_ticker, tickers, rolling_window = 10, n_pca = 5):

  lagged_returns = returns.shift(1)
  features = pd.DataFrame(index = returns.index)

  #add basic statistical features
  features[f'{target_ticker}_mean'] = lagged_returns[target_ticker].rolling(rolling_window).mean()
  features[f'{target_ticker}_vol'] = lagged_returns[target_ticker].rolling(rolling_window).std()
  features[f'{target_ticker}_skew'] = lagged_returns[target_ticker].rolling(rolling_window).skew()
  features[f'{target_ticker}_kurt'] = lagged_returns[target_ticker].rolling(rolling_window).kurt()
  features[f'{target_ticker}_momentum'] = lagged_returns[target_ticker] - lagged_returns[target_ticker].rolling(rolling_window).mean()
  features[f'{target_ticker}_momentum_ratio'] = features[f'{target_ticker}_momentum'] / (features[f'{target_ticker}_vol'] + 1e-6)

  #compress redudant lag features using PCA
  lags = pd.DataFrame({f'lag_{i}': returns[target_ticker].shift(i) for i in range(1,11)}).dropna()
  pca = PCA(n_components = 5)
  lagged_pca = pd.DataFrame(pca.fit_transform(lags), index = lags.index)
  lagged_pca.columns = [f'lag_pca_{i}' for i in range(lagged_pca.shape[1])]
  features = features.join(lagged_pca)

  #add technical indicators
  features[f'{target_ticker}_rolling_mean_10'] = returns[target_ticker].rolling(10).mean()
  features[f'{target_ticker}_rolling_std_10'] = returns[target_ticker].rolling(10).std()
  features[f'{target_ticker}_volatility_ratio'] = features[f'{target_ticker}_rolling_std_10'] / (features[f'{target_ticker}_rolling_mean_10'] + 1e-6)

  #use SPY correlation and covariance
  features[f'{target_ticker}_corr'] = returns[target_ticker].rolling(10).corr(returns['SPY'])
  features[f'{target_ticker}_cov'] = returns[target_ticker].rolling(10).cov(returns['SPY'])

  #add trend and volatility features
  features[f'{target_ticker}_cumulative_return'] = returns[target_ticker].rolling(20).apply(np.sum)
  features[f'{target_ticker}_max_drawdown'] = (
      returns[target_ticker].rolling(20).apply(lambda x: np.min(x / (np.maximum.accumulate(x + 1e-6)) - 1)))

  #feature enhancment: sector wide momentum
  features['sector_momentum'] = returns.drop(columns = [target_ticker]).mean(axis = 1).shift(1)
  features['sector_volatility'] = returns.drop(columns = [target_ticker]).std(axis = 1).shift(1)
  features['sector_sharpe'] = features['sector_momentum'] / (features['sector_volatility'] + 1e-6)

  #add temporal regularization
  features[f'{target_ticker}_ewm_mean'] = returns[target_ticker].ewm(span = 10).mean().shift(1)
  features[f'{target_ticker}_ewm_std'] = returns[target_ticker].ewm(span = 10).std().shift(1)

  #implement VIX and TNX [macroeconomic indicators (volatility and rates)]
  vix_raw = yf.download("^VIX", start = "2008-01-01", end = "2025-12-31")['Close']
  tnx_raw = yf.download("^TNX", start = "2008-01-01", end = "2025-12-31")['Close']

  vix_znorm = (vix_raw - vix_raw.rolling(60).mean()) / (vix_raw.rolling(60).std() + 1e-6)
  tnx_znorm = (tnx_raw - tnx_raw.rolling(60).mean()) / (tnx_raw.rolling(60).std() + 1e-6)
  features['vix_zscore'] = vix_znorm.shift(1)
  features['tnx_zscore'] = tnx_znorm.shift(1)

  #implement a volatility regime switch
  features['market_vol_regime'] = (features['vix_zscore'] > 1).astype(int)

  #add lagged macros
  for lag in range(1, 6):
      features[f'vix_zscore_lag{lag}'] = features['vix_zscore'].shift(lag)
      features[f'tnx_zscore_lag{lag}'] = features['tnx_zscore'].shift(lag)

  #add interaction feature
  for lag in range(1, 6):
      features[f'{target_ticker}_lag{lag}_momentum'] = returns[target_ticker].shift(lag) - returns[target_ticker].shift(lag+5)
      features[f'{target_ticker}_lag{lag}_mom_vol'] = (
          features[f'{target_ticker}_lag{lag}_momentum'] * features[f'{target_ticker}_vol'])

  #add rolling beta and market deviation
  features[f'{target_ticker}_rolling_beta'] = (
      returns[target_ticker].rolling(20).cov(returns['SPY']) /
      returns['SPY'].rolling(20).var()
  ).shift(1)

  features[f'{target_ticker}_market_dev'] = (
      returns[target_ticker] - returns['SPY']
  ).rolling(7).mean().shift(1)

  #add sector interaction signals (correlation with other signals)
  for other in tickers:
      if other != target_ticker:
          features[f'{target_ticker}_{other}_corr'] = (
              returns[target_ticker].rolling(15).corr(returns[other]))

  features[f'{target_ticker}_returns_1d'] = returns[target_ticker].shift(1)

  #add time based features
  features['month'] = features.index.month
  features['week_of_year'] = features.index.isocalendar().week.astype(int)
  features['day_of_week'] = features.index.dayofweek
  features['is_month_start'] = features.index.is_month_start.astype(int)
  features['is_month_end'] = features.index.is_month_end.astype(int)

  return features.dropna()

#create a function for the training and evaluating of model
def train_evaluate_model(features, target, target_ticker, sector_params, verbose = False):
  target = target.dropna()
  features = features.dropna()
  common_index = features.index.intersection(target.index)
  X = features.loc[common_index]
  y = target.loc[common_index]

  #drop multicollinear features
  corr_matrix = X.corr().abs()
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=  1).astype(bool))
  to_drop = [columns for columns in upper.columns if any(upper[columns] > 0.95)]
  X = X.drop(columns = to_drop)

  #add a dummy variable
  sector_dummies = pd.get_dummies([target_ticker] * len(X), prefix = 'sector')
  features = pd.concat([features, pd.DataFrame(sector_dummies.values,
                                               index = X.index, columns = sector_dummies.columns)], axis = 1)

  #scale the features
  scaler = StandardScaler()
  X_scaled = pd.DataFrame(scaler.fit_transform(X), index = X.index, columns = X.columns)

  #train, test split with OOT sample
  X_train = X_scaled.loc['2010-01-01':'2020-12-31']
  X_test = X_scaled.loc['2021-01-01':'2025-12-31']
  y_train = y.loc['2010-01-01':'2020-12-31']
  y_test = y.loc['2021-01-01':'2025-12-31']

  #initialize and fit the model
  params = sector_params.get(target_ticker, {"max_depth": 10, "num_leaves": 64})
  model = LGBMRegressor(n_estimators = 1000, learning_rate = 0.01, verbose = -1,
                        subsample = 0.8, colsample_bytree = 0.8, reg_alpha = 0.05,
                        reg_lambda = 0.5, random_state = 42, **params)
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_test)

  #drop noise with feature importance
  importances_df = pd.DataFrame({'Feature': X_scaled.columns, 'Importance': model.feature_importances_})
  important_feats = importances_df[importances_df['Importance'] > np.percentile(importances_df['Importance'], 65)]
  X_scaled_top = X_scaled[important_feats['Feature']]

  #re-evaluate using top features
  X_train_P = X_scaled_top.loc['2010-01-01':'2020-12-31']
  X_test_P = X_scaled_top.loc['2021-01-01':'2025-12-31']
  y_train = y.loc['2010-01-01':'2020-12-31']
  y_test = y.loc['2021-01-01':'2025-12-31']

  #fit model again
  model.fit(X_train_P, y_train)

  #predict again
  y_pred = model.predict(X_test_P)

  #compute the evaluation metrics
  r2 = r2_score(y_test, y_pred)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)

  print(f"LightGBM -> R^2: {r2:.4f}, RMSE: {rmse:.6f}, MAE: {mae:.6f}")

  return {
        'model': model,
        'r2': r2,
        'rmse': rmse,
        'mae': mae,
        'y_test': y_test,
        'y_pred': pd.Series(y_pred, index = y_test.index),
        'X_test': X_test
        'importances_df': importances_df
    }

#create a function to plot feature importances
def plot_feature_importances(importances_df, target_ticker):

  #sort by importance and keep only the top 20 features
  top_feats = importances_df.sort_values(by = 'Importance', ascending = False).head(20)
  plt.figure(figsize = (10, 6))
  plt.barh(top_feats['Feature'], top_feats['Importance'], color = 'teal')
  plt.title(f'Top 20 Feature Importances: {target_ticker}')
  plt.xlabel('Importance')
  plt.ylabel('Features')
  plt.legend()
  plt.show()

#create a function to plot the predicted VS actual
def plot_actual_vs_predicted(y_test, y_pred, target_ticker, title_note = ""):
    plt.figure(figsize = (12, 6))
    plt.plot(y_test.index, y_test.values, label = 'Actual', color = 'crimson', alpha = 0.7, linewidth = 2)
    plt.plot(y_test.index, y_pred.values, label = 'Predicted', color = 'royal blue', alpha = 0.7, linewidth = 2)
    plt.title(f'{target_ticker} - LightGBM Predicted Returns', fontsize = 14)
    plt.xlabel('Date')
    plt.ylabel('Returns')
    plt.grid(True)
    plt.legend()
    plt.show()